In [1]:
import numpy as np
from Assignments.Third.Model import MyFFLM, cross_entropy
import re
from tqdm import tqdm

In [2]:
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

with open("lotrFotr.txt","r",encoding='utf-8') as file:
    raw_txt = file.read()


stripped_txt_lotr = raw_txt.replace('\n', ' ').replace('\t', ' ').replace('\r', ' ')\
    .replace(';', '').replace(':', '').replace('  ', ' ').replace('.', '').replace(',', '').replace('"', '').lower()
stripped_txt_lotr = re.sub(r'[^\w\s]', '', stripped_txt_lotr).split()
stripped_txt_lotr = stripped_txt_lotr[:len(stripped_txt_lotr)//40]

# with open("mobydick.txt","r",encoding='utf-8') as file:
#     raw_txt = file.read()
#
# stripped_txt_mb = raw_txt.replace('\n', ' ').replace('\t', ' ').replace('\r', ' ')\
#     .replace(';', '').replace(':', '').replace('  ', ' ').replace('.', '').replace(',', '').lower().split()

In [3]:
vocab = list(set(stripped_txt_lotr))
vocab.sort()
vocab_dict = {}
for i, word in enumerate(vocab):
    identity_vector = np.zeros(len(vocab))
    identity_vector[i] = 1
    vocab_dict[word] = identity_vector
print(len(vocab))

1331


In [9]:
# Need to split up bc of ram issues, batches is the answer I think
memory_depth = 15
dataX = np.zeros((len(stripped_txt_lotr) - memory_depth, memory_depth, len(vocab_dict)), dtype='float32')
dataY = np.zeros((len(stripped_txt_lotr) - memory_depth, len(vocab_dict)), dtype='int16')
for i in range(len(stripped_txt_lotr) - memory_depth):
    x = np.zeros((memory_depth, len(vocab_dict)))
    for j in range(memory_depth):
        x[j] = vocab_dict[stripped_txt_lotr[i+j]]
    dataX[i] = x
    dataY[i] = vocab_dict[stripped_txt_lotr[i+memory_depth]]

In [10]:
reversed_dict = {}
for key in vocab_dict:
    reversed_dict[np.argmax(vocab_dict[key])] = key
for x in dataX[0]:
    print(reversed_dict[np.argmax(x)])
    print(x)
print(reversed_dict[np.argmax(dataY[0])])
print(dataY[0], np.argmax(dataY[0]))

three
[0. 0. 0. ... 0. 0. 0.]
rings
[0. 0. 0. ... 0. 0. 0.]
for
[0. 0. 0. ... 0. 0. 0.]
the
[0. 0. 0. ... 0. 0. 0.]
elvenkings
[0. 0. 0. ... 0. 0. 0.]
under
[0. 0. 0. ... 0. 0. 0.]
the
[0. 0. 0. ... 0. 0. 0.]
sky
[0. 0. 0. ... 0. 0. 0.]
seven
[0. 0. 0. ... 0. 0. 0.]
for
[0. 0. 0. ... 0. 0. 0.]
the
[0. 0. 0. ... 0. 0. 0.]
dwarflords
[0. 0. 0. ... 0. 0. 0.]
in
[0. 0. 0. ... 0. 0. 0.]
their
[0. 0. 0. ... 0. 0. 0.]
halls
[0. 0. 0. ... 0. 0. 0.]
of
[0 0 0 ... 0 0 0] 826


In [11]:
v_len = len(vocab)
data_set_length = len(stripped_txt_lotr) - memory_depth
model = MyFFLM(v_len, 128, learning_rate=0.1, memory_depth=memory_depth)
loss = 0

# for batch in range(data_set_length // batch_size):
for epoch in range(5):
    epoch_progress = tqdm(total=dataX.shape[0], desc="epoch progression {}/{}, loss: {}, accuracy: 0".format(epoch+1, 15, loss))
    loss = 0
    epoch_accuracy = 0
    for k, (x, y) in enumerate(zip(dataX, dataY)):
        y_pred = model.forward(x)
        model.backprop(y, y_pred)
        loss += 0.005*(cross_entropy(y, y_pred) - loss)
        if y[np.argmax(y_pred)] == 1:
            epoch_accuracy += 1
        if k % 100 == 0:
            epoch_progress.set_description(desc="epoch progression {}/{}, loss: {}, accuracy: {}".format(epoch+1, 15, loss, epoch_accuracy/(k+1)), refresh=True)
        epoch_progress.update()
    epoch_progress.close()



epoch progression 1/15, loss: 6.796637217392807, accuracy: 0.06281243207998262: 100%|██████████| 4663/4663 [03:20<00:00, 23.30it/s]  
epoch progression 2/15, loss: 6.6355133351331546, accuracy: 0.07476635514018691: 100%|██████████| 4663/4663 [03:18<00:00, 23.46it/s]
epoch progression 3/15, loss: 6.683204215409571, accuracy: 0.07324494675070636: 100%|██████████| 4663/4663 [03:20<00:00, 23.22it/s] 
epoch progression 4/15, loss: 2.590095283455936, accuracy: 0.09900990099009901:   4%|▎         | 165/4663 [00:26<03:11, 23.45it/s]

KeyboardInterrupt: 

In [7]:
vectors = model.embedding_layer.weights
np.savetxt('lotrvec1.tsv', vectors, delimiter='\t')


In [8]:
em_weights = model.embedding_layer.weights
np.savetxt('emw1.csv', vectors, delimiter=',')
h_weights = model.hidden_layer.weights
np.savetxt('hiddenw1.csv', vectors, delimiter=',')

out_weights = model.output_layer.weights
np.savetxt('outw1.csv', vectors, delimiter=',')
